Find SP by name in all SQL Server instances and databases

In [1]:
# Find SP by name in all SQL Server instances and databases
import pyodbc
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas

##########################################
# sp_name = 'sp_WhoIsActive'
##########################################

sp_name = input('Enter SP name')
print (sp_name)

query1 = """select name from sys.databases where database_id > 4 and state != 6 order by 1""" 
print(f"=== Searching for Stored Procedure '{sp_name}' in all SQL Server instances ===")

# Loop over all SQL Server Instances
for server in ('USSQL', 'BSQL', 'KSSQL', 'ZSSQL', 'WSSQL', 'SQLCLUST','TESTINGSQL', 'BIDEV16','DWH16-AZURE', 'BMANBAS', 'FSQL2'):
    # server parameters
    database = 'DBA' 
    username = 'naya5' 
    password = '5732@45$DF3!'
    conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    
    conn = pyodbc.connect(conn_string)
    conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

    # Get databases list
    print(f'Connecting to {server}')
    engine = sa.create_engine(conn_url)
    df1 = pd.read_sql_query(query1, engine)
    
    # Loop over all databases
    for i, database in df1.iterrows():
        db_name = database["name"]
        # print (db_name)
        query2 = """SELECT count(1) as count from [""" + db_name + """].sys.procedures where name = '""" + sp_name + """';""".format(input)
        # print(query2)
        df2 = pd.read_sql_query(query2, engine)
        sp_exists = (df2["count"].iloc[0])
        
        if sp_exists > 0:
            print (f"*** Found '{sp_name}' in Server: {server}, in Database: {db_name} ***")
       
print("=== Done searching ===")   



KeyboardInterrupt: Interrupted by user